<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/%E3%83%9B%E3%83%83%E3%83%88%E3%83%9A%E3%83%83%E3%83%91%E3%83%BC%E3%83%93%E3%83%A5%E3%83%BC%E3%83%86%E3%82%A3_%E3%83%AA%E3%83%A9%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.7EIboWgz2e/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.9AuY7SkpKe/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.VDt146a8QP/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [101]:
#環境変数設定
TARGET = '千葉県'
TARGET_URL = 'https://beauty.hotpepper.jp/CSP/kr/freewordSearch/?freeword=%E5%8D%83%E8%91%89&genreAlias=relax&searchGender=ALL&pn='

# ※千葉リラク
# TARGET_URL = 'https://beauty.hotpepper.jp/CSP/kr/freewordSearch/?freeword=%E5%8D%83%E8%91%89&genreAlias=relax&searchGender=ALL&pn='

NUM_TO_FETCH = 1000
STORES_PER_PAGE = 20
DRIVER_WAIT_TIME = 2

# 各種ファイル保存先
ACCU_LIST_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_hotpepper/riraku/accumulated_list_hotpepperbeauty_chiba.csv'
SUBMIT_FILE_PATH ='/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_hotpepper/riraku/result_list_hotpepperbeauty_chiba.xlsx'

#取得データ格納先
result_dict = { 
    '店名': [],
    '住所': [],
    '電話番号':[],
    'URL':[],
}

In [102]:
#標準ライブラリ
import os
import math
import json
import re

#サードパーティライブラリ
import pandas as pd
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from logging import (
    getLogger, 
    StreamHandler, 
    DEBUG, 
    INFO, 
    Formatter, 
    config,
)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
#関数定義
def add_store_url(elements, store_urls, count_skip, accu_set):
    for a_element in a_elements:
        store_url = a_element.get_attribute('href')
        store_name = a_element.text.strip()
        if store_name not in accu_set:
            store_urls.append(store_url)
        else:
            count_skip += 1
    return store_urls, count_skip

In [104]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
    "profile.managed_default_content_settings.plugins": 2,  # プラグインの無効化
})

driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

In [105]:
#カスタムロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_hotpepper/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(TARGET_URL)
      logger.debug(f'{TARGET}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{TARGET}：トップページへ移動出来ませんでした: [e]')
      raise

2023-04-17 08:50:27,638 [DEBUG] main: 千葉県：トップページへ移動しました


In [106]:
#店舗リストの全ページ数を導出
span_element = driver.find_element(By.CSS_SELECTOR,".numberOfResult")
num_all_stores = int(span_element.text)
num_all_pages = math.ceil(num_all_stores / STORES_PER_PAGE)

#積み上げリストから既存取得の店舗URLを抽出
try:
  df_accu = pd.read_csv(ACCU_LIST_PATH, usecols=[0])
  accu_list = df_accu.iloc[:, 0].values.tolist()
  accu_set = set(accu_list)
except FileNotFoundError:
  accu_set = set()

# 目標件数を満たすまでstore_urlを取得
# 1ページ目
try:
  a_elements = driver.find_elements(By.CSS_SELECTOR,".slcHead a")
  store_urls = []
  count_skip = 0
  store_urls, count_skip = add_store_url(a_elements, store_urls, count_skip, accu_set)
  logger.debug(
    f'{TARGET}：店舗リスト取得 (取得数:{len(store_urls)}, 重複数：{count_skip})'
    )
except  Exception as e:
  logger.error(f'{TARGET}：店舗リストへ移動出来ませんでした: [e]')
  raise

#2ページ目以降
i = 1
while len(store_urls) < NUM_TO_FETCH:
  i += 1
  page_url = f'{TARGET_URL}{i}'
  try:
    if i < num_all_pages:
      count_skip = 0
      print(page_url)
      driver.get(page_url)
      # リラク
      a_elements = driver.find_elements(By.CSS_SELECTOR,".slcHead a")
      store_urls, count_skip = add_store_url(a_elements, store_urls, count_skip, accu_set)
      logger.debug(
        f'{TARGET}：店舗リスト{i}取得 (累計:{len(store_urls)}, 重複：{count_skip})'
      )
    else:
      logger.info(f'{TARGET}：取得出来る店舗情報がありません')
  except Exception as e:
    logger.error(f'{TARGET}：店舗リスト{i}へ移動出来ませんでした: [e]')

logger.info(f'{TARGET}：店舗URL取得完了:{len(store_urls)}個')  
print(store_urls)


2023-04-17 08:50:29,279 [DEBUG] main: 千葉県：店舗リスト取得 (取得数:0, 重複数：20)
https://beauty.hotpepper.jp/CSP/kr/freewordSearch/?freeword=%E5%8D%83%E8%91%89&genreAlias=relax&searchGender=ALL&pn=2
2023-04-17 08:50:39,580 [DEBUG] main: 千葉県：店舗リスト2取得 (累計:0, 重複：20)
https://beauty.hotpepper.jp/CSP/kr/freewordSearch/?freeword=%E5%8D%83%E8%91%89&genreAlias=relax&searchGender=ALL&pn=3
2023-04-17 08:50:46,943 [DEBUG] main: 千葉県：店舗リスト3取得 (累計:0, 重複：20)
https://beauty.hotpepper.jp/CSP/kr/freewordSearch/?freeword=%E5%8D%83%E8%91%89&genreAlias=relax&searchGender=ALL&pn=4
2023-04-17 08:50:51,054 [DEBUG] main: 千葉県：店舗リスト4取得 (累計:0, 重複：20)
https://beauty.hotpepper.jp/CSP/kr/freewordSearch/?freeword=%E5%8D%83%E8%91%89&genreAlias=relax&searchGender=ALL&pn=5
2023-04-17 08:50:54,766 [DEBUG] main: 千葉県：店舗リスト5取得 (累計:0, 重複：20)
https://beauty.hotpepper.jp/CSP/kr/freewordSearch/?freeword=%E5%8D%83%E8%91%89&genreAlias=relax&searchGender=ALL&pn=6
2023-04-17 08:50:59,059 [DEBUG] main: 千葉県：店舗リスト6取得 (累計:0, 重複：20)
https://beauty.hotp

In [ ]:
#店舗個別ページにて情報取得
page_num = 0
try:
  for store_url in store_urls:
    try:
        driver.get(store_url)
        page_num += 1
        logger.debug(f'{TARGET}：店舗ページ{page_num}へ移動しました')
    except Exception as e:
        logger.error(f'{TARGET}：店舗ページへ移動出来ませんでした: [e]')
    try:
        #店舗名/住所取得
        store_name = driver.find_element(By.CSS_SELECTOR,".detailTitle").text
        address = driver.find_element(By.CSS_SELECTOR,".w620").text
        if address=="番号を表示":
          td_elements = driver.find_elements(By.CSS_SELECTOR,".vaThT td")
          address = td_elements[1].text
        #電話番号ページ遷移
        #正規表現を使って、URLの?以降を取り除く
        tmp = re.sub(r'\?.*', '', store_url)
        tel_url = f'{tmp}tel/'
        try:
            driver.get(tel_url)
            logger.debug(f'{TARGET}：店舗電話番号ページ{page_num}へ移動しました')
        except Exception as e:
            logger.error(f'{TARGET}：店舗電話番号ページへ移動出来ませんでした: [{e}]')
        #電話番号取得
        phone_number = driver.find_element(By.CSS_SELECTOR, ".fs16").text
        #結果格納
        result_dict['店名'].append(store_name)
        result_dict['電話番号'].append(phone_number)
        result_dict['住所'].append(address)
        result_dict['URL'].append(store_url)
    except Exception as e:
        logger.error(f'{TARGET}：{store_name}の店舗情報取得に失敗しました: {store_url}')
        #結果格納
        result_dict['店名'].append(store_name)
        result_dict['電話番号'].append("ー")
        result_dict['住所'].append(address)
        result_dict['URL'].append(store_url)
except KeyboardInterrupt:
  logger.info(f'{TARGET}：処理を中断します')
finally:
  df_result = pd.DataFrame(result_dict)
  if not os.path.isfile(SUBMIT_FILE_PATH):
    df_result.to_excel(SUBMIT_FILE_PATH, header=True, index=False)
  else:
    with pd.ExcelWriter(SUBMIT_FILE_PATH, mode='a', if_sheet_exists='new') as writer:
      df_result.to_excel(writer)
  df_result.to_csv(ACCU_LIST_PATH, mode='a', header=True, index=False)
  logger.info(f'{TARGET}：実行終了しデータを保存しました')
  df_accu

df_result = pd.DataFrame(result_dict)
df_result

2023-04-17 08:58:47,539 [DEBUG] main: 千葉県：店舗ページ1へ移動しました
2023-04-17 08:58:48,448 [DEBUG] main: 千葉県：店舗電話番号ページ1へ移動しました
2023-04-17 08:58:51,072 [DEBUG] main: 千葉県：店舗ページ2へ移動しました
2023-04-17 08:58:51,839 [DEBUG] main: 千葉県：店舗電話番号ページ2へ移動しました
2023-04-17 08:58:54,554 [DEBUG] main: 千葉県：店舗ページ3へ移動しました
2023-04-17 08:58:55,172 [DEBUG] main: 千葉県：店舗電話番号ページ3へ移動しました
2023-04-17 08:58:58,010 [DEBUG] main: 千葉県：店舗ページ4へ移動しました
2023-04-17 08:58:58,903 [DEBUG] main: 千葉県：店舗電話番号ページ4へ移動しました
2023-04-17 08:59:03,356 [DEBUG] main: 千葉県：店舗ページ5へ移動しました
2023-04-17 08:59:05,693 [DEBUG] main: 千葉県：店舗電話番号ページ5へ移動しました
2023-04-17 08:59:09,673 [DEBUG] main: 千葉県：店舗ページ6へ移動しました
2023-04-17 08:59:11,284 [DEBUG] main: 千葉県：店舗電話番号ページ6へ移動しました
2023-04-17 08:59:14,007 [DEBUG] main: 千葉県：店舗ページ7へ移動しました
2023-04-17 08:59:17,962 [DEBUG] main: 千葉県：店舗電話番号ページ7へ移動しました
2023-04-17 08:59:19,761 [DEBUG] main: 千葉県：店舗ページ8へ移動しました
2023-04-17 08:59:21,596 [DEBUG] main: 千葉県：店舗電話番号ページ8へ移動しました
2023-04-17 08:59:23,647 [ERROR] main: 千葉県：トリートメントサロンPono【ポノ】の店舗情報取得に失敗しま

In [ ]:
# # 提出ファイル用出力
# if not os.path.isfile(SUBMIT_FILE_PATH):
#     df_result.to_excel(SUBMIT_FILE_PATH, header=True, index=False)
# else:
#   with pd.ExcelWriter(SUBMIT_FILE_PATH, mode='a', if_sheet_exists='new') as writer:
#     df_result.to_excel(writer)

# # 積み上げリスト保存用出力
# df_result.to_csv(ACCU_LIST_PATH, mode='a', header=True, index=False)

In [ ]:
# 積み上げリスト確認
df_accu = pd.read_csv(ACCU_LIST_PATH)
df_accu